# In Memoriam, a shift into the post-democratic
## Student: Marlon Tabone
### Model Training

This notebook details the processes of training models on datasets to generate a model which is able to output text pertinent to Daphne's subjects and style of writing in her blog "Running Commentary"

In [15]:
# importing necessary libraries, packages for text preprocessing 
import nltk
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import 	WordNetLemmatizer

#importing layers,models and optimizers from Keras tensorflow

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
import codecs
import collections

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/marlontabone/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading the dataset on which to train our models...

In [16]:
# loading .txt file 
path = 'datasets/0.txt'
text = open(path).read().lower() # reading text
print('Corpus length:', len(text)) #printing length of text
print(text)
nltk.download('punkt')

Corpus length: 39924
and he gets a good answer too also shouldnt the education minister a university lecturer be quoting from source rather than just googling facebookfriendly quotes but hell be hard pushed to find a source for that one the clue is in the use of revenge as a verb one takes revenge one does not revenge but im sure the man who pretends to love english so as to pick up the votes of undiscriminating pepe types knows that alreadysuch rubbish  even in the education ministry  stephen bonellos cartoon in the sunday times today  i love it  britains supreme court has ruled today that the referendum result on britains exit from the european union does not give the government a mandate and that the matter must be decided by parliamentthis is a timely reminder that in representative democracies there can be no government by mob rule  and a referendum is essentially not an exercise in democracy but in mob rulethe referendum is of great political significance but the act of parliamen

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marlontabone/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

A function to clean our loaded dataset and make it free from punctation. The idea is to reduce the noise so that our model can focus on the actual words. 

In [17]:
def clean_doc(doc):

     clean_words= []

     words = word_tokenize(text) #tokenise text into words
    
     for word in words: #for every word
           word = word.strip(string.punctuation) #remove punctuation
           #eliminate specified words
           if len(word)>=1 and word.isdigit()==False and word not in ("äì","\n","Äô","\n\n",'\u2009','\xa0','n’t',',','.','\n  \n\n  \n',' \n\n','“','\n\n  \n',':','-'):
                    word = word.lower() #make everything lowercase 
                    clean_words.append(word) #append the clean free from punctuation list
                        
     return clean_words # return list which is now our new vocabulary

token=clean_doc(text)
print(token) 

['and', 'he', 'gets', 'a', 'good', 'answer', 'too', 'also', 'shouldnt', 'the', 'education', 'minister', 'a', 'university', 'lecturer', 'be', 'quoting', 'from', 'source', 'rather', 'than', 'just', 'googling', 'facebookfriendly', 'quotes', 'but', 'hell', 'be', 'hard', 'pushed', 'to', 'find', 'a', 'source', 'for', 'that', 'one', 'the', 'clue', 'is', 'in', 'the', 'use', 'of', 'revenge', 'as', 'a', 'verb', 'one', 'takes', 'revenge', 'one', 'does', 'not', 'revenge', 'but', 'im', 'sure', 'the', 'man', 'who', 'pretends', 'to', 'love', 'english', 'so', 'as', 'to', 'pick', 'up', 'the', 'votes', 'of', 'undiscriminating', 'pepe', 'types', 'knows', 'that', 'alreadysuch', 'rubbish', 'even', 'in', 'the', 'education', 'ministry', 'stephen', 'bonellos', 'cartoon', 'in', 'the', 'sunday', 'times', 'today', 'i', 'love', 'it', 'britains', 'supreme', 'court', 'has', 'ruled', 'today', 'that', 'the', 'referendum', 'result', 'on', 'britains', 'exit', 'from', 'the', 'european', 'union', 'does', 'not', 'give', '

Implementing Lemmatization...

In [18]:
wd = WordNetLemmatizer()
newDict = []
for w in token:
        newDict.append(wd.lemmatize(w))
vocab_size = len(token)
print(vocab_size)
print(newDict)

6869
['and', 'he', 'get', 'a', 'good', 'answer', 'too', 'also', 'shouldnt', 'the', 'education', 'minister', 'a', 'university', 'lecturer', 'be', 'quoting', 'from', 'source', 'rather', 'than', 'just', 'googling', 'facebookfriendly', 'quote', 'but', 'hell', 'be', 'hard', 'pushed', 'to', 'find', 'a', 'source', 'for', 'that', 'one', 'the', 'clue', 'is', 'in', 'the', 'use', 'of', 'revenge', 'a', 'a', 'verb', 'one', 'take', 'revenge', 'one', 'doe', 'not', 'revenge', 'but', 'im', 'sure', 'the', 'man', 'who', 'pretend', 'to', 'love', 'english', 'so', 'a', 'to', 'pick', 'up', 'the', 'vote', 'of', 'undiscriminating', 'pepe', 'type', 'know', 'that', 'alreadysuch', 'rubbish', 'even', 'in', 'the', 'education', 'ministry', 'stephen', 'bonellos', 'cartoon', 'in', 'the', 'sunday', 'time', 'today', 'i', 'love', 'it', 'britain', 'supreme', 'court', 'ha', 'ruled', 'today', 'that', 'the', 'referendum', 'result', 'on', 'britain', 'exit', 'from', 'the', 'european', 'union', 'doe', 'not', 'give', 'the', 'gov

Creating text sequences, from which we will later extract out seed_sentences

In [19]:
sequence_len=30 #specifying length of sequences 
seq=[]
for i in range(0,len(token)-sequence_len):
    seq.append(token[i:i+sequence_len])

In [20]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(seq)
sequence = tokenizer.texts_to_sequences(seq)

In [21]:
print(sequence[2])

print(len(sequence))
import numpy as np
arr=np.array(sequence)

X, Y = arr[:,:-1], arr[:,-1]
Y=to_categorical(Y,num_classes=vocab_size)

[729, 5, 234, 451, 187, 67, 726, 1, 725, 66, 5, 723, 1835, 31, 1834, 44, 722, 231, 105, 75, 1832, 1831, 1830, 33, 1829, 31, 448, 730, 3, 453]
6839


The printed sequence above is representative of the number of time that the word appears in the text. This will determined the wrods chances of being chose as a next word when generating text

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding

seq_length = X.shape[1]

model = Sequential()
model.add(Embedding(vocab_size, 150, input_length=seq_length))
model.add(LSTM(256, return_sequences=True)) #most of the times parameters here were 512 instead of 256
model.add(LSTM(256)) 
model.add(layers.Dropout(0.2)) # dropout layer 
model.add(Dense(256, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 29, 150)           1030350   
_________________________________________________________________
lstm_2 (LSTM)                (None, 29, 256)           416768    
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 6869)              1765333   
Total params: 3,803,555
Trainable params: 3,803,555
Non-trainable params: 0
____________________________________________

In [24]:
model.compile(loss='categorical_crossentropy',
optimizer='adam', metrics=['accuracy']) #we want to measure how accurate our training is to the dataset

In [25]:
model.fit(X, Y, batch_size=80, epochs=150)

Train on 6839 samples
Epoch 1/150
6839/6839 [==============================] - 28s 4ms/sample - loss: 7.1976 - accuracy: 0.0566
Epoch 2/150
6839/6839 [==============================] - 24s 3ms/sample - loss: 6.4190 - accuracy: 0.0594
Epoch 3/150
6839/6839 [==============================] - 25s 4ms/sample - loss: 6.2644 - accuracy: 0.0594
Epoch 4/150
6839/6839 [==============================] - 23s 3ms/sample - loss: 6.1484 - accuracy: 0.0613
Epoch 5/150
6839/6839 [==============================] - 24s 3ms/sample - loss: 6.0523 - accuracy: 0.0680
Epoch 6/150
6839/6839 [==============================] - 23s 3ms/sample - loss: 5.9785 - accuracy: 0.0731
Epoch 7/150
6839/6839 [==============================] - 24s 4ms/sample - loss: 5.8586 - accuracy: 0.0752
Epoch 8/150
6839/6839 [==============================] - 25s 4ms/sample - loss: 5.7262 - accuracy: 0.0798
Epoch 9/150
6839/6839 [==============================] - 24s 3ms/sample - loss: 5.6069 - accuracy: 0.0885
Epoch 10/150
6839/6839 [

6839/6839 [==============================] - 24s 4ms/sample - loss: 1.3320 - accuracy: 0.6089
Epoch 78/150
6839/6839 [==============================] - 23s 3ms/sample - loss: 1.3189 - accuracy: 0.6190
Epoch 79/150
6839/6839 [==============================] - 23s 3ms/sample - loss: 1.2967 - accuracy: 0.6137
Epoch 80/150
6839/6839 [==============================] - 23s 3ms/sample - loss: 1.2812 - accuracy: 0.6105
Epoch 81/150
6839/6839 [==============================] - 23s 3ms/sample - loss: 1.2219 - accuracy: 0.6375
Epoch 82/150
6839/6839 [==============================] - 23s 3ms/sample - loss: 1.2188 - accuracy: 0.6397
Epoch 83/150
6839/6839 [==============================] - 23s 3ms/sample - loss: 1.2061 - accuracy: 0.6372
Epoch 84/150
6839/6839 [==============================] - 24s 3ms/sample - loss: 1.1675 - accuracy: 0.6464
Epoch 85/150
6839/6839 [==============================] - 24s 3ms/sample - loss: 1.1439 - accuracy: 0.6600
Epoch 86/150
6839/6839 [==========================

In [40]:
### Length of extracted word sequences
maxlen = 30

# We sample a new sequence every `step` words
step = 1

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up words)
next_words = []

for i in range(0, len(token) - maxlen, step):
    sentences.append(token[i: i + maxlen])
    next_words.append(token[i + maxlen])

Generating above the seed sentences to initiate the generation of text in the function below.... 

In [41]:
#importing some packages 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from nltk.tokenize import word_tokenize
import nltk
import random as rand
nltk.download('punkt')

from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize.treebank import TreebankWordDetokenizer


tokenised_text = sentences[rand.randrange(len(sentences))]
seed_text = TreebankWordDetokenizer().detokenize(tokenised_text)

#using this function to predict the next word
def generate_seq(model,tokenizer,seq_length,seed_text,n_words):

     result = []
     in_text =seed_text
     for i in range(n_words):
          encoded = tokenizer.texts_to_sequences([in_text])[0]
          encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
          yhat = model.predict_classes(encoded)
          out_word = ''
          for word,index in tokenizer.word_index.items():
               if index==yhat:
                  out_word = word
                  break
          in_text+=' ' + out_word
          result.append(word) #append each predicted word to result array
     return ' '.join(result) #join the list in the end to form the text


generated = generate_seq(model, tokenizer, seq_length, seed_text, 100) #predict 100 words
print("Seed: "+seed_text)
print(seed_text + generated)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marlontabone/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Seed: meant to scrutinise and not in the service of the publicthe people hounding jason azzopardi and hanging around his office door are not journalists but labour party employees and the
meant to scrutinise and not in the service of the publicthe people hounding jason azzopardi and hanging around his office door are not journalists but labour party employees and thesooner we get this straight the better they dont even make any attempt at pretending that what they are doing is journalism and they are not after the truth or the facts but are there as agents of the governing party to harass politicians of the rival opposition party that is not an exercise in democracy and freedom of expression but the precise oppositewhich brings me to another related point the labour party is in government it is in power it should understand that the tactics it used when in opposition look really very ugly now it is one thing


In [42]:
model.save("blocks.h5")
print("Saved model to disk") #finally save model to disk, so we can access it in bot.py

Saved model to disk
